In [7]:
import cupy as cp
from cupyx.scipy.ndimage import zoom
import glob
import nibabel as nib
from nibabel.processing import resample_from_to
from nibabel.orientations import axcodes2ornt, ornt_transform, apply_orientation, inv_ornt_aff
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

In [8]:
def resample_mri(img, new_spacing=(1.0, 1.0, 1.0), order=3, reorient=True):
    """
    Resamples a NIfTI MRI image to the desired spacing using spline interpolation and saves the result.
    
    Parameters:
    - input_file (str): Path to the input NIfTI file.
    - output_file (str): Path to save the resampled NIfTI file.
    - new_spacing (tuple of float): The desired spacing in each dimension (z, y, x). Default is (1.0, 1.0, 1.0).
    - order (int): The order of the spline interpolation. Default is 3 for cubic interpolation.
    """
    
    # Load the MRI image using nibabel
    input_image = img.get_fdata()
    original_spacing = img.header.get_zooms()
    
    # Convert the input image to a CuPy array
    input_image_cp = cp.array(input_image)
    
    # Calculate the zoom factors for each dimension
    zoom_factors = tuple(os / ns for os, ns in zip(original_spacing, new_spacing))
    
    # Perform the resampling using the zoom function
    resampled_image_cp = zoom(input_image_cp, zoom_factors, order=order)
    new_affine = np.copy(img.affine)
    scale_factors = np.array(new_spacing) / np.array(original_spacing)
    new_affine[:3, :3] *= scale_factors
    
    # Convert the resampled image back to a NumPy array
    resampled_image = cp.asnumpy(resampled_image_cp)
    
    if reorient:
        # Determine the original orientation
        original_ornt = nib.orientations.io_orientation(new_affine)
        
        # Determine the transformation to RAI
        rai_ornt = axcodes2ornt(('R', 'A', 'I'))
        transform = ornt_transform(original_ornt, rai_ornt)
        
        # Apply the transformation to the image data
        resampled_image_rai = apply_orientation(resampled_image, transform)
        
        # Create a new affine matrix for the RAI-oriented image
        new_affine = new_affine @ inv_ornt_aff(transform, resampled_image.shape)
        
        # Save the resampled and reoriented image to a new NIfTI file
        new_img = nib.Nifti1Image(resampled_image_rai, new_affine)
        return new_img
    else:
        return nib.Nifti1Image(resampled_image, new_affine)
    

In [9]:
def z_score_mri_exclude_roi(mri_img, mask_img, output_file):
    """
    Performs z-score normalization on an MRI image, using statistics computed from the image
    excluding the region defined by a mask.
    
    Parameters:
    - mri_file (str): Path to the input MRI NIfTI file.
    - mask_file (str): Path to the NIfTI file containing the ROI mask.
    - output_file (str): Path to save the z-score normalized NIfTI file.
    """
    
    mri_data = mri_img.get_fdata()
    mask_data = mask_img.get_fdata()
    
    # Convert the MRI data and mask to CuPy arrays
    mri_data_cp = cp.array(mri_data)
    mask_data_cp = cp.array(mask_data)
    
    # Create a mask where the ROI is included (mask is 1 where ROI is included, 0 elsewhere)
    roi_mask_cp = mask_data_cp > 0  # ROI mask
    
    # Create an exclusion mask where the ROI is excluded
    exclusion_mask_cp = cp.logical_not(roi_mask_cp)  # Exclude ROI
    
    # Calculate the mean and standard deviation excluding the ROI
    mri_mean = cp.mean(mri_data_cp[exclusion_mask_cp])
    mri_std = cp.std(mri_data_cp[exclusion_mask_cp])
    
    # Perform z-score normalization on the entire MRI data
    z_score_normalized_cp = (mri_data_cp - mri_mean) / mri_std
    
    # Convert the result back to a NumPy array
    z_score_normalized = cp.asnumpy(z_score_normalized_cp)
    
    # Create a new NIfTI image with the normalized data and save it
    new_img = nib.Nifti1Image(z_score_normalized, mri_img.affine, mri_img.header)
    nib.save(new_img, output_file)
    
    return new_img

In [10]:
def match_roi_to_image(roi_img, ref_img):
    """
    Matches an ROI image to a reference image by resampling and aligning the ROI to the reference.

    Parameters:
    - roi_img (nibabel.Nifti1Image): The ROI NIfTI image to be aligned.
    - ref_img (nibabel.Nifti1Image): The reference NIfTI image to match the ROI against.

    Returns:
    - matched_roi_img (nibabel.Nifti1Image): The resampled and aligned ROI image.
    """
    
    # Resample the ROI to match the reference image using nibabel
    matched_roi_img = resample_from_to(roi_img, ref_img)

    return matched_roi_img

In [11]:
path_data = glob.glob("/mnt/data/psteinmetz/neotex/to_process/152")

In [12]:
for data in path_data:
    print(data)
    image_path_sub = glob.glob(
        f'{data}/RawVolume/*dyn1_bias_corrected_1_resampled_Bspline_zscore_without_tumour.nii.gz'
    )

    mask_path = glob.glob(f'{data}/RoiVolume/Bounding_box.nii.gz')
    if image_path_sub == []:
        print('problem image')
        pass
    elif mask_path == []:
        if glob.glob(f'{data}/RoiVolume/*C1_dyn*') != []:
            print('to_check')
            mask_path =glob.glob(f'{data}/RoiVolume/*C1_dyn*') 
            image_sub = nib.load(image_path_sub[0])
            mask = nib.load(mask_path[0])
            matched_mask = match_roi_to_image(mask, image_sub)

            interp_im = resample_mri(image_sub)
            resampled_mask = resample_mri(matched_mask, order=1)
            final = z_score_mri_exclude_roi(interp_im, resampled_mask, data + '/RawVolume/substracted_resampled_bspline_zscore_without_tumor.nii.gz')
            nib.save(resampled_mask, data + '/RoiVolume/New_C1_volume_resampled_NN.nii.gz')
            
        elif glob.glob(f'{data}/RoiVolume/*C1_' + data.split('/')[-1] + '*') != []:
            print('to_check')
            mask_path = glob.glob(f'{data}/RoiVolume/*C1_' + data.split('/')[-1] + '*')
            
            image_sub = nib.load(image_path_sub[0])
            mask = nib.load(mask_path[0])
            matched_mask = match_roi_to_image(mask, image_sub)

            interp_im = resample_mri(image_sub)
            resampled_mask = resample_mri(matched_mask, order=1)
            final = z_score_mri_exclude_roi(interp_im, resampled_mask, data + '/RawVolume/substracted_resampled_bspline_zscore_without_tumor.nii.gz')
            nib.save(resampled_mask, data + '/RoiVolume/New_C1_volume_resampled_NN.nii.gz')
        else:
            print('problem mask')
            pass

    else:
        image_sub = nib.load(image_path_sub[0])
        mask = nib.load(mask_path[0])
        matched_mask = match_roi_to_image(mask, image_sub)

        interp_im = resample_mri(image_sub)
        resampled_mask = resample_mri(matched_mask, order=1)
        final = z_score_mri_exclude_roi(interp_im, resampled_mask, data + '/RawVolume/dyn1_resampled_bspline_zscore_without_tumor.nii.gz')
        nib.save(resampled_mask, data + '/RoiVolume/New_bb_dyn1_resampled.nii.gz')

/mnt/data/psteinmetz/neotex/to_process/152


In [ ]:
path_data[0].split('/')[-1]

In [ ]:
im = final.get_fdata()
ma = resampled_mask.get_fdata()

In [ ]:
plt.close('all')
plt.imshow(im[:, :, 100], interpolation='none', cmap='grey')
plt.imshow(ma[:, :, 100], alpha=0.25*(ma[:, :, 100]>0))
plt.show()

In [ ]:
plt.close('all')
plt.imshow(im[:, 240, :], interpolation='none', cmap='grey')
plt.imshow(ma[:, 240, :], alpha=0.25*(ma[:, 240, :]>0))
plt.show()